### purpose

train lfmm on complex sims

### notes

The first sbatching of jobs failed because of ballooning memory issues. I eventually found out this was because I wasn't specifying K in the LEA::genetic.offset function, whereas I had specified k for the non-multivariate sims and they ran fine. After specifying K, the multivariate jobs ran fine

In [1]:
from pythonimports import *

import MVP_10_train_lfmm2_offset as mvp10
import MVP_summary_functions as mvp

lview, dview = get_client(cluster_id='1694090917-49bo', profile='lotterhos')

outerdir = '/home/b.lind/offsets/run_20220919_tutorial'
tdir = f'{outerdir}/tutorial'

mvp.latest_commit()
session_info.show()

36 36
#########################################################
Today:	September 07, 2023 - 10:03:52 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 9493334506a1f2926fba0734fa74ba2f87194611  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 30 09:41:00 2023 -0400

Current commit of MVP_offsets:
commit 34fec27ce185eb670eb16124a5255c9d4964ee70  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Sep 5 11:47:40 2023 -0400
#########################################################



In [2]:
mvp_dir = '/home/b.lind/code/MVP-offsets/01_src'
# path to lfmm offset training script
# training_script = op.join(mvp_dir, 'MVP_climate_outlier_process_lfmm.R')
training_script = f'{mvp_dir}/MVP_complex_sims_process_lfmm.R'

op.exists(training_script)

True

In [3]:
outerdir = '/home/b.lind/offsets/run_20220919_tutorial'

indir = makedir(op.join(outerdir, 'lfmm2/lfmm_infiles'))
outdir = makedir(op.join(outerdir, f'lfmm2/lfmm_outfiles'))
shdir = makedir(op.join(outerdir, 'lfmm2/lfmm_shfiles'))

In [4]:
run = op.basename(outerdir)

run

'run_20220919_tutorial'

In [5]:
inds = pd.read_table(op.join(tdir, 'My_Individuals.txt'))

inds

,ind_index,x,y,phenotype1_mat,phenotype2_MTWetQ,phenotype3_MTDQ,phenotype4_PDM,phenotype5_PwarmQ,phenotype6_PWM,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM,subpopID,lon,lat
0,33,0.406184,0.233272,0.737967,-0.191421,0.142992,-0.131894,-0.452497,-0.541651,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821,25,5.0,3.0
1,34,0.425697,0.837158,-0.274081,-0.428332,-0.012880,0.221058,0.229296,0.066597,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680,85,5.0,9.0
2,44,0.671673,0.581744,-0.676326,0.086629,-0.470671,0.218813,0.260098,-0.103465,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511,57,7.0,6.0
3,45,0.017466,0.329922,-0.283221,-0.403942,-0.333718,-0.200554,-0.345915,-0.410001,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979,31,1.0,4.0
4,46,0.069744,0.121221,0.057616,-0.303201,0.731600,0.195679,-0.367110,0.196296,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751,11,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8575,0.117890,0.278449,-0.070241,-0.367056,-0.061876,-0.308041,-0.468111,-0.508108,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126,22,2.0,3.0
996,8580,0.464067,0.792001,-0.050435,-0.342693,0.128758,0.107181,-0.061796,-0.081682,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439,75,5.0,8.0
997,8582,0.158645,0.682325,0.354088,0.493751,0.064744,-0.243331,-0.164916,-0.244379,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783,62,2.0,7.0
998,8585,0.894104,0.757964,-0.009027,0.424444,-0.575177,-0.514613,0.586778,0.266235,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799,79,9.0,8.0


# main

### mvp10.create_current_envfiles

In [6]:
inds[[col for col in inds if col.startswith('env')]]

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751
...,...,...,...,...,...,...
995,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126
996,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439
997,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783
998,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799


In [7]:
# mvp10.create_current_envfiles

# read in previous envfile
envdata = inds[[col for col in inds if col.startswith('env')]].copy()

lfmm_envfile = op.join(indir, 'tutorial_lfmm_env_ntraits-6.txt')
envdata.to_csv(lfmm_envfile, sep='\t', index=False, header=False)

envdata

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751
...,...,...,...,...,...,...
995,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126
996,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439
997,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783
998,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799


### mvp10.create_poplabels

In [8]:
# mvp10.create_poplabels

inds = pd.read_table(op.join(tdir, 'My_Individuals.txt'))

display(inds.head())

poplabel_file = op.join(indir, f'tutorial_poplabels.txt')

inds['subpopID'].to_csv(poplabel_file, sep='\t', index=False, header=False)

!wc -l $poplabel_file

,ind_index,x,y,phenotype1_mat,phenotype2_MTWetQ,phenotype3_MTDQ,phenotype4_PDM,phenotype5_PwarmQ,phenotype6_PWM,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM,subpopID,lon,lat
0,33,0.406184,0.233272,0.737967,-0.191421,0.142992,-0.131894,-0.452497,-0.541651,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821,25,5.0,3.0
1,34,0.425697,0.837158,-0.274081,-0.428332,-0.012880,0.221058,0.229296,0.066597,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680,85,5.0,9.0
2,44,0.671673,0.581744,-0.676326,0.086629,-0.470671,0.218813,0.260098,-0.103465,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511,57,7.0,6.0
3,45,0.017466,0.329922,-0.283221,-0.403942,-0.333718,-0.200554,-0.345915,-0.410001,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979,31,1.0,4.0
4,46,0.069744,0.121221,0.057616,-0.303201,0.731600,0.195679,-0.367110,0.196296,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751,11,1.0,2.0


1000 /home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_poplabels.txt


### mvp10.create_transplant_envfiles

In [9]:
envdata.shape

(1000, 6)

In [10]:
envdata.head()

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751


In [11]:
pop_envdata = inds.groupby('subpopID')[[col for col in inds if col.startswith('env')]].apply(np.mean)

pop_envdata

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
subpopID,,,,,,
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...
96,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592


In [12]:
# mvp10.create_transplant_envfiles

garden_files = []

for subpop_id, env_vals in pop_envdata.iterrows():
    X_new = envdata.copy()
    
    # overwrite env vals with single value of transplant env
    for col in X_new:
        X_new[col] = env_vals[col]
        
    lfmm_envfile_new = lfmm_envfile.replace('.txt',
                                           f'_{str(subpop_id).zfill(3)}.txt')
    
    X_new.to_csv(lfmm_envfile_new, sep='\t', index=False, header=False)
    
    garden_files.append(lfmm_envfile_new)    
    
len(garden_files)

100

In [13]:
X_new

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
1,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
2,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
3,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
4,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
...,...,...,...,...,...,...
995,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
996,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
997,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
998,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116


### create_adaptive_and_neutral_files

In [14]:
# get marker sets (from ../00_create_files_for_GF_and_RONA_train_GF.ipynb)
snps = pd.read_table(op.join(tdir, 'SNPs.txt'), delim_whitespace=True)

snps['idx'] = snps.index + 1

snps.index = snps.mutname.tolist()

print(snps.shape, '\n')

display(snps)

marker_sets = {
    'all' : snps.mutname.tolist(),
    'neutral' : snps[snps.LG > 10]['mutname'].tolist(),
    'adaptive' : snps[snps.causal == True]['mutname'].tolist()
}

for marker_set, loci in marker_sets.items():
    print(marker_set, len(loci))

(26371, 6) 



,LG,pos_pyslim,mutname,a_freq_subset,causal,idx
1-8,1,8,1-8,0.0175,False,1
1-27,1,27,1-27,0.0120,False,2
1-34,1,34,1-34,0.0370,False,3
1-81,1,81,1-81,0.0330,False,4
1-97,1,97,1-97,0.0170,False,5
...,...,...,...,...,...,...
20-999742,20,999742,20-999742,0.0125,False,26367
20-999764,20,999764,20-999764,0.0120,False,26368
20-999823,20,999823,20-999823,0.0120,False,26369
20-999845,20,999845,20-999845,0.0135,False,26370


all 26371
neutral 13285
adaptive 90


In [15]:
snps.idx.loc[loci]

1-7998         209
1-29086        743
1-29881        765
1-35368        917
1-36525        950
             ...  
10-457403    11960
10-463933    12126
10-477349    12480
10-478574    12527
10-481150    12588
Name: idx, Length: 90, dtype: int64

In [16]:
loci_files = []

for marker_set, loci in marker_sets.items():
    if marker_set == 'all':
        continue
    
    loci_indices = snps.idx.loc[loci]
    
    break
    
pd.DataFrame(loci_indices)

,idx
11-500069,13087
11-500093,13088
11-500129,13089
11-500130,13090
11-500132,13091
...,...
20-999742,26367
20-999764,26368
20-999823,26369
20-999845,26370


In [17]:
loci_files = []
for marker_set, loci in marker_sets.items():

    loci_indices = snps['idx'].loc[loci]
    loci_df = pd.DataFrame(loci_indices)
    
    print(marker_set, len(loci))
    display(loci_df)
    
    loci_file = op.join(indir, f'tutorial_{marker_set}_loci.txt')
    loci_df.to_csv(loci_file, index=False, header=False)

    loci_files.append(loci_file)

all 26371


,idx
1-8,1
1-27,2
1-34,3
1-81,4
1-97,5
...,...
20-999742,26367
20-999764,26368
20-999823,26369
20-999845,26370


neutral 13285


,idx
11-500069,13087
11-500093,13088
11-500129,13089
11-500130,13090
11-500132,13091
...,...
20-999742,26367
20-999764,26368
20-999823,26369
20-999845,26370


adaptive 90


,idx
1-7998,209
1-29086,743
1-29881,765
1-35368,917
1-36525,950
...,...
10-457403,11960
10-463933,12126
10-477349,12480
10-478574,12527


### mvp10.create_shfiles

In [18]:
loci_files

['/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_all_loci.txt',
 '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_neutral_loci.txt',
 '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_adaptive_loci.txt']

In [19]:
# mvp10.create_shfiles(lfmm_envfiles, poplabel_file, garden_files, locus_files, thresh=36)

neutral_file, adaptive_file = loci_files[-2:]

print(adaptive_file, '\n', neutral_file)

# how to know when I'm done creating commands = num_gardens * num_marker_sets
expected_jobcount = len(garden_files) * (len(loci_files))  # hard-coded 1 as "all" loci below

expected_jobcount

/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_adaptive_loci.txt 
 /home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_neutral_loci.txt


300

In [20]:
training_script

'/home/b.lind/code/MVP-offsets/01_src/MVP_complex_sims_process_lfmm.R'

In [21]:
# creating a symlink from srcfile to dst didn't work - trying now by saving with sep=' '
srcfile = op.join(tdir, 'Genotypes.txt')

dst = op.join(indir, 'Genotypes.lfmm')

df = pd.read_table(srcfile, delim_whitespace=True, header=None)

df.to_csv(dst, sep=' ', index=False, header=False)

```R

# I DID NOT DO THIS THE SECOND TIME AROUND - I USED THE SYMLINK FILE

# try saving the genotypes with lfmm in R in case that helps reduce memory requirements
# saving as I did above made lfmm want > 1Tb memory for a single command

library(LEA)

geno = read.table('/home/b.lind/offsets/run_20220919_tutorial/tutorial/Genotypes.txt', header=F)

write.lfmm(geno, '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/Genotypes.lfmm')
```

In [22]:
thresh = 36

jobcount = 0  # to know when I'm done iterating
cmds = []  # list of training commands to parallelize for a given job
shfiles = []
for garden_file in pbar(garden_files):
    garden = op.basename(garden_file).split("_")[-1].split('.')[0]  # subpopID
    
    for cand_file, marker_set in zip(['None', adaptive_file, neutral_file], ['all', 'adaptive', 'neutral']):
        output_file = op.join(outdir, f'tutorial_lfmm_offsets_ntraits-6_{marker_set}_{garden}.txt')

        cmd = ' '.join(['Rscript',
                        training_script,
                        dst,  # dst from previous cell (Genotypes.lfmm)
                        lfmm_envfile,
                        garden_file,
                        poplabel_file,
                        output_file,
                        cand_file])
        cmds.append(cmd)
        jobcount += 1

        if len(cmds) == thresh or jobcount == expected_jobcount:
            batch_num = str(len(shfiles)).zfill(3)
            job = f'{run}_tutorial_lfmm_batch_{batch_num}'

            # determine amount of memory required (with cushion)
            if len(cmds) == thresh:
                mem = '175000M' 
            else:
                val = (len(cmds) * 4900) + 5000
                mem = f'{val}M'

            # write commands to a file that I can `cat` to GNU parallel
            cmd_file = op.join(indir, f'{job}.txt')
            with open(cmd_file, 'w') as o:
                o.write('\n'.join(cmds))

            # where to write sh text
            shfile = op.join(shdir, f'{job}.sh')

            # what text to write to sh
            text = f'''#!/bin/bash
#SBATCH --job-name={job}
#SBATCH --time=03:00:00
#SBATCH --mem={mem}
#SBATCH --partition=short
#SBATCH --nodes=1
#SBATCH --cpus-per-task=36
#SBATCH --output={job}_%j.out
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL

source $HOME/.bashrc  # assumed that conda init is within .bashrc

module load parallel

conda activate MVP_env_R4.0.3

# train lfmm2 and calculate offset
cat {cmd_file} | parallel -j {len(cmds)} --progress --eta

conda activate mvp_env

cd {mvp_dir}

# re run any failed jobs from the cmd_file
python MVP_watch_for_failure_of_train_lfmm2_offset.py tutorial {shfile} {outerdir} {len(cmds)}

'''
            with open(shfile, 'w') as o:
                o.write(text)
            shfiles.append(shfile)

            cmds = []  # reset

len(shfiles)

100%|███████████████| 100/100 [00:00<00:00, 176.01it/s]


9

In [23]:
print(read(shfiles[0], lines=False))

#!/bin/bash
#SBATCH --job-name=run_20220919_tutorial_tutorial_lfmm_batch_000
#SBATCH --time=03:00:00
#SBATCH --mem=175000M
#SBATCH --partition=short
#SBATCH --nodes=1
#SBATCH --cpus-per-task=36
#SBATCH --output=run_20220919_tutorial_tutorial_lfmm_batch_000_%j.out
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL

source $HOME/.bashrc  # assumed that conda init is within .bashrc

module load parallel

conda activate MVP_env_R4.0.3

# train lfmm2 and calculate offset
cat /home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/run_20220919_tutorial_tutorial_lfmm_batch_000.txt | parallel -j 36 --progress --eta

conda activate mvp_env

cd /home/b.lind/code/MVP-offsets/01_src

# re run any failed jobs from the cmd_file
python MVP_watch_for_failure_of_train_lfmm2_offset.py tutorial /home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_shfiles/run_20220919_tutorial_tutorial_lfmm_batch_000.sh /home/b.lind/offsets/run_20220919_tutorial 36




In [24]:
pids = sbatch(shfiles)

pids

sbatching: 100%|███████████████| 9/9 [00:00<00:00, 86778.70it/s]


['38489307',
 '38489308',
 '38489309',
 '38489310',
 '38489311',
 '38489312',
 '38489313',
 '38489314',
 '38489315']